<a href="https://colab.research.google.com/github/zahidhamidi/ML-Project/blob/main/Unit_test_RBM_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from tqdm.auto import tqdm
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_excel("/content/sample_data/robust_test_dataset.xlsx")

In [6]:
def text_extraction(text,topics):

  # Define the topics as flexible terms (subwords or whole words)
  # topics = ["gas shows" , "oil show", "overpressure", "fluid communication" , "gas bearing" , "h2s" , "co2" , "stuck pipe" , "pressure build up", "sand production"]


  # Replace hyphens with blank spaces in all items in the topics list
  topics = [topic.replace('-', ' ') for topic in topics]
  text = [text.replace('-', ' ') for topic in topics]


  # Initialize a list to store selected sentences
  selected_sentences = []

  # Initialize a flag to check for both topic terms and negation terms
  both_detected = False

  # Tokenize the text into sentences using NLTK's sent_tokenize
  sentences = sent_tokenize(text)

  # Initialize flags to check for both topic terms and negation terms
  topic_found = False
  negation_detected = False

  if len(sentences) == 1:
    selected_sentences.append(text)
    # print(0)

  else:

    for sentence in sentences:

      # remove apostrophes from string
      sentence = sentence.replace("'", "")
      sentence = sentence.replace(",", "")

      # Tokenize the sentence into words
      lemmatized_words = word_tokenize(sentence)

      # Join the lemmatized words back into a sentence
      lemmatized_sentence = ' '.join(lemmatized_words)

      # Initialize an empty pattern string
      pattern_string = r'(?i)(?:(?<=\s)|^)'

      for topic_pattern in topics:
        words = topic_pattern.split()

        if len(words) >= 2:

          # Split the topic words by spaces and hyphens and create a pattern to match them
          # topic_pattern = re.compile(r'(?i)\b' + r'\b|\b'.join(re.escape(topic) for topic in topics) + r'\b')

          # Create a regex pattern to match the multi-word topic with any characters in between
          pattern_string = r'(?i)\b' + r'\s*[,.\s]*\s*'.join(re.escape(word) for word in words) + r'\b'

          # Compile the regex pattern
          topic_pattern = re.compile(pattern_string, re.IGNORECASE)


          if re.search(topic_pattern, lemmatized_sentence.lower()):
                topic_found = True
                break

        elif len(words) != 0:
            single_pattern = rf'(?i)(?:(?<=\s)|^)({re.escape(words[0])})(?=\s|$)'
            if re.search(single_pattern, lemmatized_sentence.lower()):
                topic_found = True
                break

      # Check if the lemmatized sentence contains negation terms
      if re.search(negation_terms, lemmatized_sentence.lower()):
          negation_detected = True
      else:
          negation_detected = False

      counter = 0
      # If a topic term is found and either no negation terms are found or negation_detected is False, append the sentence
      if (topic_found is True) and (negation_detected is True):
          selected_sentences.append(lemmatized_sentence)
          both_detected = True
          # print(1)
          break

      elif (topic_found is True) and (negation_detected is False):
          selected_sentences.append(lemmatized_sentence)
          both_detected = False
          counter = counter + 1
          # print(2)

          if counter >1:
            break
          break

      # else:
      #   selected_sentences.append(sentences)



    # If no sentence with both topic and negation terms is found, append the first sentence without a negation term
    if topic_found is False:
        selected_sentences.append(text)

  # print("Selected Sentence: ", selected_sentences)
  # print("Topic word found: ",topic_found)
  # print("Negation word found: ",negation_detected)

  return selected_sentences








In [4]:
def unit_test(text,topic):
  selected = text_extraction(text,topic)
  print(selected)

In [9]:
finalized_text = []

topic = "gas show"

for doc in tqdm(df["doc_text_original"]):
  selected = text_extraction(doc,topic)

  if len(selected) == 1:
    finalized_text.append(selected[0])

  else:
    sentence = ".".join(selected)
    finalized_text.append(sentence)




  # Add the selected_sentences as a new column named 'selected_sentence'
  # df['selected_sentence'] = selected

 # Add the selected_sentences as a new column named 'selected_sentence'
df['selected_sentence'] = finalized_text


  0%|          | 0/2161 [00:00<?, ?it/s]

In [14]:
# Define the negation terms pattern with case-insensitivity
negation_terms = r'(?i)(?:(?<=\s)|^)(no|non|unlikely|none|not|nor|without|lack|rather)(?=\s|$|\b)'


# Determine the prediction (0 or 1) based on the presence of negation terms in selected_sentence
for index, row in df.iterrows():
    selected_sentence = row['selected_sentence']

    if selected_sentence is not None:
        # Check if the lemmatized sentence contains negation terms
        if re.search(negation_terms, selected_sentence.lower()):
            prediction = 0  # Negation terms found, set the prediction to 0
        else:
            prediction = 1  # No negation terms found, set the prediction to 1
    else:
        prediction = 0  # Handle the case where selected_sentence is None

    df.at[index, 'predicted'] = prediction

# Print the DataFrame with the 'label' column
df[['doc_text_original', 'selected_sentence', 'label_code', 'predicted']]

,doc_text_original,selected_sentence,label_code,predicted
0,"'Shows: (0il, Gas, â€˜ Show of gas while drill...","'Shows: (0il, Gas, â€˜ Show of gas while drill...",1,1.0
1,"""VI. Reservoirs: Sand reservoirs are present i...",A thin 8 foot sand in the lower portipn of the...,1,1.0
2,"'Shows: (0il, Gas, â€˜ Show of gas while drill...","'Shows: (0il, Gas, â€˜ Show of gas while drill...",1,1.0
3,"""Drilling shale as above with _ occasional. st...","""Drilling shale as above with _ occasional. st...",1,1.0
4,'74 | -~ | 73| Drilling shale as above with â€...,'74 | -~ | 73| Drilling shale as above with â€...,1,1.0
...,...,...,...,...
2156,21 21A 22 22A 23 23A 24 24A 25 25A 26 26A 27 2...,21 21A 22 22A 23 23A 24 24A 25 25A 26 26A 27 2...,1,0.0
2157,"""used to calculate the geochemical Altbough th...",The 1761.8m & till has quite a low value ident...,1,1.0
2158,"""used to calculate the geochemical Altbough th...",The 1761.8m & till has quite a low value ident...,1,1.0
2159,lower levels of these unknown compounds is not...,sufficiently understood as yet it is not possi...,1,0.0


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

accuracy_values = []
precision_values = []
recall_values = []
f1_values = []

In [16]:
# Assuming you have loaded your DataFrame 'df' with the columns 'label_code' and 'predicted_label_code'
y_true = df['label_code']
# y_pred = df['prediction_negex']
y_pred = df['predicted']

# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Calculate additional evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# Append metric values to the respective lists
accuracy_values.append(accuracy)
precision_values.append(precision)
recall_values.append(recall)
f1_values.append(f1)

print(accuracy, precision, recall, f1)

0.8056455344747802 0.6698925524299644 0.7274762085166873 0.6885628638957744


In [18]:
df

,id,result_id,source_name,source_type,name,basin,field_name,operator,longitude_decimal_degree,longitude_degree,...,article_title,document_id,document_title,page_number,dataset,label,label_code,predicted,doc_text_original,selected_sentence
0,0,b4be2e6a-12a7-4cae-bdf3-dcb7329a46d3,2001-07-02 00:00:00,CENTRAL GRABEN,ELDFISK,PHILLIPS PETROLEUM COMPANY NORWAY,NaN,3.0,12.0,8.78,...,Public Databases,b038d824-7552-4ea8-ae73-02bf824f5e64,2_7-1_COMPLETION_REPORT_08280_02-07-01-X_PB-70...,44,gas shows,NaN,1,1.0,"'Shows: (0il, Gas, â€˜ Show of gas while drill...","'Shows: (0il, Gas, â€˜ Show of gas while drill..."
1,1,08a60fa2-b879-4516-b909-b35ed18a0bb1,2001-08-25 00:00:00,NORTHERN NORTH SEA,BALDER,ESSO EXPLORATION AND PRODUCTION NORWAY A/S,NaN,2.0,29.0,29.47,...,Public Databases,5d2af66b-6ae0-41ea-866e-b3e1e6542e8f,25_8-1__WELL__25-08-01_PB-706-0711.PDF,52,gas shows,NaN,1,1.0,"""VI. Reservoirs: Sand reservoirs are present i...",A thin 8 foot sand in the lower portipn of the...
2,2,2959a4a5-bafc-40b0-a6e3-006388096a20,2001-07-02 00:00:00,CENTRAL GRABEN,ELDFISK,PHILLIPS PETROLEUM COMPANY NORWAY,NaN,3.0,12.0,8.78,...,Public Databases,228a6137-3de8-4465-ae62-97367e97f34a,2_7-1_COMPLETION_REPORT_84843_02-07-01-X_PB-70...,44,gas shows,NaN,1,1.0,"'Shows: (0il, Gas, â€˜ Show of gas while drill...","'Shows: (0il, Gas, â€˜ Show of gas while drill..."
3,3,114ebc61-eb32-4e52-9513-606accd4e4cf,2001-11-02 00:00:00,CENTRAL GRABEN,VALHALL,AMOCO NORWAY OIL COMPANY,NaN,3.0,27.0,1.67,...,Public Databases,9ccfafd8-170f-4974-a1be-e727b0239531,2_11-1_COMPLETION_REPORT_82700_02-11-01_PB-706...,82,gas shows,NaN,1,1.0,"""Drilling shale as above with _ occasional. st...","""Drilling shale as above with _ occasional. st..."
4,4,3b026c1e-d3f6-4bb8-aa6c-d0d4abafdd2f,2001-11-02 00:00:00,CENTRAL GRABEN,VALHALL,AMOCO NORWAY OIL COMPANY,NaN,3.0,27.0,1.67,...,Public Databases,9ccfafd8-170f-4974-a1be-e727b0239531,2_11-1_COMPLETION_REPORT_82700_02-11-01_PB-706...,49,gas shows,NaN,1,1.0,'74 | -~ | 73| Drilling shale as above with â€...,'74 | -~ | 73| Drilling shale as above with â€...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156,96,75fa2376-4a24-411d-ba91-8a398018e980,SWAN 3-ST1,BONAPARTE,SWAN,BHP PETROLEUM PTY LTD,124.493210,NaN,NaN,NaN,...,Public Databases,0bec2748-41ba-4f21-8c30-65db23551d9f,Swan_3_3ST1_WCR_Interpretive_I00010109_Bonapar...,112,oil show,NaN,1,0.0,21 21A 22 22A 23 23A 24 24A 25 25A 26 26A 27 2...,21 21A 22 22A 23 23A 24 24A 25 25A 26 26A 27 2...
2157,97,bb8150d5-7207-4e11-9c87-16b460e43610,LORIKEET-1,BONAPARTE,LORIKEET,BHP PETROLEUM PTY LTD,125.619232,NaN,NaN,NaN,...,Public Databases,5c445c91-3503-40f6-bfa3-87edb6bb6dfb,Lorikeet_1_WCR_Interpretative_Data_I00028852_B...,87,oil show,NaN,1,1.0,"""used to calculate the geochemical Altbough th...",The 1761.8m & till has quite a low value ident...
2158,98,bb8150d5-7207-4e11-9c87-16b460e43610,LORIKEET-1 ST-1,BONAPARTE,LORIKEET,BHP PETROLEUM PTY LTD,125.619232,NaN,NaN,NaN,...,Public Databases,5c445c91-3503-40f6-bfa3-87edb6bb6dfb,Lorikeet_1_WCR_Interpretative_Data_I00028852_B...,87,oil show,NaN,1,1.0,"""used to calculate the geochemical Altbough th...",The 1761.8m & till has quite a low value ident...
2159,99,888c2d05-ad64-4aed-b722-271d6d7cef02,MONTARA-1,BONAPARTE,MONTARA,BHP PETROLEUM PTY LTD,124.532920,NaN,NaN,NaN,...,Public Databases,9e575914-9f9f-44bb-a42b-ddec1da52b29,WCR_Interpretive_Data_I00028891.pdf,384,oil show,NaN,1,0.0,lower levels of these unknown compounds is not...,sufficiently understood as yet it is not possi...


In [20]:
# Assuming 'df' is your DataFrame
df.to_excel("dataset_gb.xlsx", index=False)